In [6]:
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [14]:
cursor=db.companies.aggregate([
    {'$match': 
     {'$and': [ 
        {'category_code': {'$in': ["web","software","games_video"]}},
        {'founded_year':{'$gte':2008}},
        {'deadpooled_year': None },
        {"offices.latitude":{'$ne': None}},
        {"number_of_employees":{'$ne': None}},
    ]}},
    { '$unwind': "$offices" },
    { '$project' : {
        "name" : 1,
        "category_code": 1,
        "founded_year": 1,
        "description":1,
        "money_raised":1,
        "number_of_employees":1,
        "offices.latitude": 1,
        "offices.longitude":1
    }}])

Comp=list(cursor)

In [16]:
cursor=db.companies.aggregate([
    {'$match': 
     {'$and': [ 
        {'category_code': {'$in': ["web","software","games_video"]}},
        {'founded_year':{'$gte':2008}},
        {'deadpooled_year': None },
        {"offices.latitude":{'$ne': None}},
        {"number_of_employees":{'$ne': None}},
    ]}},
    { '$unwind': "$offices" },
    { '$project' : {
        "name" : 1,
        "category_code": 1,
        "founded_year": 1,
        "description":1,
        "money_raised":1,
        "number_of_employees":1,
        #"city": { '$arrayElemAt': [ "$offices", 4 ] },
        #"countycode": { '$arrayElemAt': [ "$offices", 6 ] },
        "latitude": { '$arrayElemAt': [ "$offices", 7 ] },
        "longitude": { '$arrayElemAt': [ "$offices", 8 ] }
    }}])

Comp=list(cursor)

OperationFailure: $arrayElemAt's first argument must be an array, but is object

In [15]:
import pandas as pd

df = pd.DataFrame(Comp)
df.head()

,_id,category_code,description,founded_year,name,number_of_employees,offices
0,52cdef7c4bab8bd675297e30,software,Private Academic Networks,2008,GoingOn,40,"{'latitude': 37.782263, 'longitude': -122.392142}"
1,52cdef7c4bab8bd675297f3f,games_video,,2008,YouBeQB,8,"{'latitude': 47.597965, 'longitude': -122.151158}"
2,52cdef7c4bab8bd675297f76,games_video,Digital Entertainment and Media Platform,2008,Crunchyroll,50,"{'latitude': 37.781265, 'longitude': -122.393229}"
3,52cdef7c4bab8bd675297fec,web,Tech Support Community,2013,Fixya,30,"{'latitude': 37.566879, 'longitude': -122.323895}"
4,52cdef7c4bab8bd675298038,games_video,,2009,alluc,7,"{'latitude': 53.707739, 'longitude': 10.023246}"


In [55]:
def toGeoJSON(array):
    return [array["latitude"],array["longitude"]]
    

def convertCompany(company):
    return {
        "name": company["name"],
        "category_code": company["category_code"],
        "founded_year":company["founded_year"],
        "description":company["description"],
        "money_raised":company["total_money_raised"],
        "number_of_employees"
        "position": toGeoJSON(company["offices"])}

In [66]:
geoCompanies = list(map(lambda r: convertCompany(r),cursor))
geoCompanies

[]